# Examples of Lifting maps for Weil Representations #

Note that this file requires a working installation of the "fqm_weil" package in the SageMath environment this notebook is running in. 

The purpose of this notebook is to provide algorithms for computing lifting maps given in the paper 
Stromberg, F, "On liftings of modular forms and Weil representations". 

Currently the only explicitly verified example in this file corresponds to the example given in the paper.

In [ ]:
import fqm_weil
from fqm_weil import *
from fqm_weil.modules import *
from fqm_weil.modules.finite_quadratic_module.finite_quadratic_module_base import *
from fqm_weil.modules.finite_quadratic_module.finite_quadratic_module_ambient import *
from fqm_weil.modules.finite_quadratic_module.finite_quadratic_module_ambient import _A
from fqm_weil.modules.weil_module.weil_module import WeilModule,WeilModuleElement
from fqm_weil.modules.weil_module.utils import _entries
from fqm_weil.modules.utils import factor_matrix_in_sl2z
from fqm_weil.modular.utils import cusp_normalisers_and_stabilisers, exp_as_zN_power
import logging
import sys
from IPython.display import Markdown as md
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
log = logging.getLogger(__name__)
log.setLevel(logging.WARNING) # Change to logging.DEBUG to see debug info

## Example 1

We consider the finite quadratic module $D = A_2^t$ with $t=1$ or alternatively given by the genus symbol '$2_1$'.

In [ ]:
D = _A(2,1)
print(D)
md(f"Genus symbol: {D.jordan_decomposition().genus_symbol()}")

In [ ]:
print("Level of D=",D.level())
group = Gamma0(D.level())
cusps = group.cusps()
cusps.sort(key= lambda x:x.denominator())
print("Cusps:")
s = ""
for i, x in enumerate(cusps):
    w = 4 / gcd(4,x.denominator()**2)
    s += f"$\\mathfrak{{a}}_{i+1}$ = ${latex(x)}$ = ({x.numerator()} : {x.denominator()}), $w_{i+1}={w}$<br>"
md(s)

We choose cusp normalising maps $A_{\mathfrak{a}}$ and cusp stabilisers $T_{\mathfrak{a}}$:

In [ ]:
normalisers_and_stabilisers = cusp_normalisers_and_stabilisers(group)
s = r""
for i, cusp in enumerate(cusps):
    A = normalisers_and_stabilisers['cusp_normalisers'][cusp]
    s +=fr"$A_{i+1} = A_{{ {latex(cusp)} }}= {latex(A)}$,&nbsp;&nbsp;"
    s +=fr"$A^{{ -1}}_{{ {latex(cusp)} }}= {latex(A**-1)}$,&nbsp;&nbsp;"
    Tp = normalisers_and_stabilisers['cusp_stabilisers'][cusp]
    s +=fr"$T_{{ {latex(cusp)} }}= {latex(Tp)}$,<br>"    
md(s)

The Weil representation associated with $D$ acts on the group algebra $\mathbb{C}[D]\simeq \mathbb{C}e_0 \oplus  \mathbb{C}e_{1/2}$
and is defined by the action on the generators S, T of $SL_2(\mathbb{Z})$

$$
\rho_{D}(T)e_{\gamma} =e(Q(\gamma))e_{\gamma}
$$
$$
\rho_{D}(S)e_{\gamma} = \frac{\sigma(D)}{\sqrt{|D|}}\sum_{\delta\in D}e(-B(\gamma,\delta))e_{\delta}
$$
Here the $\sigma$-invariant $\sigma(D)=e_8(-\text{sign}(D))=\left( \frac{t}{2} \right) e_8 (-t)=e_8(-1)$ if $t=1$.

We can represent the action with matrices in $M_2( \bf{Q}(\zeta_{8}))$ together with a scaling factor: 

In [ ]:
W = WeilModule(D)
S, T = SL2Z.gens()
md(fr"$\rho_D(T)={latex(W.matrix(T))}$ <br> $\rho_D(S)={latex(W.matrix(S))}$")


Following the notation in the paper we let $\xi(B)$ denote the top-left entry of $\rho_D(B)$ and for each of the cusp stabilisers $T_{\mathfrak{a}_i}$ we write $\xi(T_{\mathfrak{a}_i})=e(\alpha_i)$ with $\alpha_i \in [0,1)$.

In [ ]:
# It doesn't matter which element of W is used to calculate \xi
w = WeilModule(D).an_element()
s = ""
for i, cusp in enumerate(cusps):
    Tp = normalisers_and_stabilisers['cusp_stabilisers'][cusp]
    xi = w.xi(Tp)
    if xi == 1:
        alpha = 0
    else:
        n = xi.parent().gen().multiplicative_order()
        alpha = xi.list().index(1)/n # find the power of z8
    s += f"$\\alpha_{i+1}={alpha}$,  "
md(s)

From this we see that the cusps $\mathfrak{a}_0=\infty$ and $\mathfrak{a}_2=0$ correspond to the component $0$ and $\mathfrak{a}_1=1/2$ corresponds to the component $\frac{1}{2}$ since $Q(0)=0$ and $Q(1/2)=1/4$. 


In [ ]:
for x in D:
    print(f"Q({x})={D.Q(x)}")

Let $F\in\mathcal{M}_{k}(\Gamma_{0}(4),v_{\theta}^{2k})$ and assume
that $F|_{A_{i}}(\tau)=\sum b_{i}(n)q^{n}$ is the Fourier expansion
of $F$ at the cusp $\mathfrak{a}_{i}$ and let the lifted vector-valued modular form be
$f=\sum_{\alpha\in D}f_{\alpha}e_{\alpha}$ then the Fourier coefficients
of $f_{\alpha}$ are given by
\begin{eqnarray*}
c(0,n) & = & \Lambda_{\infty}(0)b_{\infty}(n) &+& \Lambda_{1/2}(0)b_{1/2}(n)  &+& \Lambda_{0}(0)b_{0}(4n), n\in \mathbb{Z}\\
c\left(1/2,n\right) & = & \Lambda_{\infty}(1/2)b_{\infty}(n) &+& \Lambda_{1/2}(1/2)b_{1/2}(n)  &+& \Lambda_{0}(1/2)b_{0}(4n), n\in \mathbb{Z}+\frac{1}{4}
\end{eqnarray*}
where
$$
\Lambda_{i}\left(\alpha\right)=\sqrt{\frac{\left|D_{c_{i}}\right|}{\left|D\right|}}\xi\left(A_{i}^{-1}\right)w_{i}\sum_{\omega\in D^{c_{i}*}\cap D_{S_{0}}^{c_{i}*}\cap\{\alpha+S_0
\}}   \Phi_{S_0, a, c}(\omega) e\left(-dQ_{c_{i}}\left(\omega\right)\right),
$$
where
$$
\Phi_{S_0, a, c}(\omega) = \sum_{\gamma \in c^{-1}(S_0)/D_c} e(-acQ(\gamma) + B(\omega,\gamma)).
$$


To compute map explicitly we need
$|D_{c_i}|=|\{x\in D\mid c_i x = 0\}|$, $\xi(A_i^{-1})$, the set $\omega\in D^{c_{i}*}\cap D_{S_{0}}^{c_{i}*}\cap \{\alpha + S_0\}$, the values of $\Phi_{S_0, a, c}(\omega)$ and $Q_{c_i}(\omega)$ for all $\omega$ in this set.

Recall that 

$D_c = \{x\in D\mid c_i x = 0\}$

$D^c = \{cx \mid x \in D \}$

$D^{c*} = \{\alpha \in D \mid cQ(\gamma)+B(\gamma,\alpha)\in \mathbb{Z}, \forall \gamma \in D_c\}
 = x_c + D^c$
where $x_2=\frac{1}{2}$ and $x_c=0$ for all other $c$. 

If $S_0$ is an isotropic subgroup of $D$ then 

$D_{S_0}^{c*} = \{\alpha \in D \mid cQ(\gamma)+B(\gamma,\alpha)\in \mathbb{Z}, \forall \gamma \in c^{-1}S_0 \cap S_0^{\perp}\}$

For simplicity we assume that $S_0 = \{0\}$, in which case $\Phi_{S_0,a,c}=1$ and $D_{S_0}^{c*} = D^{c*}$ and it is clear that $\Lambda_i(\alpha)=0$ unless $\alpha \in D^{c_i*}$

We have $c_0 = 0$, $c_1=2$ and $c_3=1$ and we can see that 

In [ ]:
s = ""
for cusp in cusps:
    c = cusp.denominator()
    s += f"$D_{c} = {list(D.kernel_subgroup(c))}$, "
    s += f"$D^{c} = {list(D.power_subgroup(c))}$, "
    s += f"$D^{{ {c} *}} = {list(D.power_subset_star(c))}$<br>"
md(s)

In [ ]:
s = ""
for i, cusp in enumerate(cusps):
    c = cusp.denominator()
    for alpha in D:
        s += fr"${alpha} \in D^{{ c_{i+1}}} = D^{{ {c} *}}: {alpha in D.power_subset_star(c)}$ <br>"
md(s)

This shows that $\Lambda_{\infty}(1/2)=\Lambda_{1/2}(0)=0$ so that 
\begin{eqnarray*}
c(0,n) & = & \Lambda_{\infty}(0)b_{\infty}(n) &+& \Lambda_{0}(0)b_{0}(4n)\\
c\left(1/2,n\right) & = & \Lambda_{0}(1/2)b_{0}(n) &+& \Lambda_{1/2}(1/2)b_{1/2}(n)  
\end{eqnarray*}

In [ ]:
s = ""
w = W(D.0)
for i, cusp in enumerate(cusps):
    Ai = normalisers_and_stabilisers['cusp_normalisers'][cusp]
    s += f"$\\xi(A_{i+1})={latex(w.xi(Ai**-1))}$<br>"
md(s)

In [ ]:
s = ""
for i, cusp in enumerate(cusps):
    ci = cusp.denominator()
    Ai = normalisers_and_stabilisers['cusp_normalisers'][cusp]
    di = Ai[1][1]
    for alpha in D.power_subset_star(ci):
        DQc = D.Q_c(ci,alpha)
        s += fr"i={i+1}: $Q_{ci}({alpha}) = {DQc},\; d_{i+1}={di}\; \Rightarrow\; e(-d_i Q_{{c_i}})= {exp_as_zN_power(D.level(),-di*DQc)}$<br>"
md(s)

We can finally conclude that 

$\Lambda_{\infty}(0) = \xi(A_{\infty}^{-1}) = 1$,

$\Lambda_{0}(0) = \sqrt{\frac{1}{2}} \cdot 4 \cdot \xi(A_{2}^{-1})  = 2\sqrt{2}\zeta_8 = 2 + 2i$,

$\Lambda_{0}(1/2) = \sqrt{\frac{1}{2}} \cdot 4 \cdot \xi(A_{2}^{-1}) = 2\sqrt{2}\zeta_8 = 2 + 2i$,

$\Lambda_{1/2}(1/2) = \sqrt{\frac{2}{2}}\xi(A_{3}^{-1}) = 1$,

and the coefficients of the lifted form are given by 

\begin{eqnarray*}
c(0,n) & = & b_{\infty}(n) &+& (2+2i) b_{0}(4n)\\
c\left(1/2,n\right) & = & (2+2i) b_{0}(n) &+& b_{1/2}(n)  
\end{eqnarray*}



For more general quadratic modules and isotropic subgroups the calculations are harder to carry out by hand 
but the functions below can be used:

In [ ]:
def Phi(D, S0,a,c,omega):
    r"""
    The function \Phi_{S_0,a,c}(\omega) as defined in Lemma 16 in [2023, Stromberg, On lifting maps...]
    
    INPUT:
    
    - ``D`` -- finite quadratic module
    - ``S0`` -- isotropic subgroup of ``D``
    - ``a`` -- integer
    - ``c`` -- integer
    - ``omega`` -- element of ``D``
    
    """
    if not S0.is_subgroup(D) or not S0.is_isotropic():
        raise ValueError("S0 must be an isotropic subgroup of D")
    if omega not in D:
        raise ValueError("omega must be an element of D")
    N = D.level()
    zN = CyclotomicField(N).gen()
    result = 0
    for gamma in (S0 / c)/D.kernel_subgroup(c):
        arg = -a*c*D.Q(gamma) + D.B(omega,gamma)
        result += exp_as_zN_power(N,arg)
    return result
        
def lifting_map_Lambdas(D, S0=None):
    r"""
    Compute the values of $\Lambda_i(\alpha)$
    
    INPUT:
    
    - ``D`` -- finite quadratic module
    - ``S0`` -- isotropic subgroup of ``D``
        
    """
    if not S0:
        S0 = D.subgroup()
    assert S0.is_isotropic()
    N = D.level()
    zN = CyclotomicField(N).gen()
    G = Gamma0(N)
    normalisers_and_stabilisers = cusp_normalisers_and_stabilisers(group)
    MN = lcm(N, 8)
    CFMN = CyclotomicField(MN)
    Lambda = {}
    W=WeilModule(D)
    for cusp in G.cusps():
        Lambda[cusp] = {}
        for alpha in D:
            Lambda[cusp][alpha] = 0
            
    for cusp in G.cusps():
        log.debug("*"*50)
        log.debug(f"cusp={cusp}")
        a = cusp.numerator()
        c = cusp.denominator()
        w =  N / gcd(N,c**2)
        Ai = normalisers_and_stabilisers['cusp_normalisers'][cusp]
        log.debug(f"c={c}")
        d = Ai[1][1]
        log.debug(f"d={d}")
        log.debug(f"Ai^-1={Ai**-1}")
        A = D.power_subset_star(c)
        log.debug(f"D^c*={A}")
        B = D.power_subset_kernel_star(c,S0)
        Dc = len(D.kernel_subgroup(c))
        AA = set(A).intersection(B)
        xi = W(D.0).xi(Ai**-1)
        for alpha in D:
            log.debug("-"*20)
            log.debug(f"alpha={alpha}")
            C = S0 + alpha
            for omega in AA.intersection(C):
                log.debug(f"c,omega={c,omega}")
                log.debug(f"d*Q_c(omega)={d, D.Q_c(c,omega)}")
                arg = -d*D.Q_c(c,omega)
                term = Phi(D, S0, a, c, omega)*exp_as_zN_power(N,arg)
                Lambda[cusp][alpha] += term
                #if alpha == 0 and a == 0:
                log.debug(f"term={omega,Phi(D,S0,a,c,omega),exp_as_zN_power(N,arg)}")
            try: 
                fac = CFMN(xi*w*(QQ(Dc)/QQ(D.order())).sqrt())
            except TypeError:
                fac = xi*w*sqrt(Dc/D)
            Lambda[cusp][alpha] *= fac
    # In the end try to convert    
    return Lambda

In [ ]:
def lambdas_to_formula(D, Lambdas):
    s = ""
    N = D.level()
    for alpha in D:
        sl = []
        for cusp in Gamma0(N).cusps():
            L = Lambdas[cusp][alpha]
#             if not L:
#                 continue
            w = N / gcd(N, cusp.denominator()**2)
            if w == 1:
                w = ""
            if L == 1:
                Lfactor = ""
            else: 
                Lfactor = fr"({latex(L)})\cdot"
            sl.append(fr"{Lfactor} b_{latex(cusp)}({w}n)")
        s +=  fr"$c({latex(alpha)}, n) = " + "+".join(sl) + "$<br>"
    return s

Test it on the example above:

In [ ]:
F = _A(2,1)
lifting_map_Lambdas(F,F.subgroup())

In [ ]:
md(lambdas_to_formula(F, lifting_map_Lambdas(F,F.subgroup())))

Test the module $A_2^{3} = q_3^{-1}$ (corresponding to modular forms on $\Gamma_0(4)$ of weight $\frac{3}{2}\pmod{2}$)

In [ ]:
F = _A(2,3)
lifting_map_Lambdas(F,F.subgroup())

## Example 2
A more complex example, also of level $4$. Consider $D=B_2 \oplus A_2^{1}$

In [ ]:
D = FiniteQuadraticModule('2^2.2_1')
s = ""
for alpha in D:
     s += fr"$Q({latex(alpha)})={latex(D.Q(alpha))}$<br>"
S0 = D.isotropic_subgroups()[1]
print("S0=",S0)
md(s)

In [ ]:
normalisers_and_stabilisers = cusp_normalisers_and_stabilisers(Gamma0(D.level()))
w = WeilModule(D).an_element()
s = ""
for i, cusp in enumerate(cusps):
    Tp = normalisers_and_stabilisers['cusp_stabilisers'][cusp]
    xi = w.xi(Tp)
    if xi == 1:
        alpha = 0
    else:
        n = xi.parent().gen().multiplicative_order()
        alpha = xi.list().index(1)/n # find the power of z8
    s += f"$\\alpha_{i+1}={alpha}$,  "
md(s)

There are three insotropic subgroups:

In [ ]:
D.isotropic_subgroups()

Lift for the subgroup $\langle 0\rangle$

In [ ]:
md(lambdas_to_formula(D, lifting_map_Lambdas(D,D.isotropic_subgroups()[0])))

Lift for the subgroup $\langle e_0\rangle$

In [ ]:
md(lambdas_to_formula(D, lifting_map_Lambdas(D,D.isotropic_subgroups()[1])))

Lift for the subgroup $\langle e_1\rangle$

In [ ]:
md(lambdas_to_formula(D, lifting_map_Lambdas(D,D.isotropic_subgroups()[2])))